<a href="https://colab.research.google.com/github/GeorgeShishkanov/university_project/blob/main/Token_classificayin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading

In [60]:
!unzip /content/FactRuEval.zip

Archive:  /content/FactRuEval.zip
replace dev.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [61]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [62]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:
import pandas as pd
import re
import csv
from datasets import Dataset

# Конвертация в csv

In [64]:
with open('/content/train.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('train.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('title', 'intro'))
        writer.writerows(lines)
with open('/content/dev.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('val.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('title', 'intro'))
        writer.writerows(lines)
with open('/content/test.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('test.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('title', 'intro'))
        writer.writerows(lines)

# Обработка датасета

In [65]:
DATA_TRAIN = pd.read_csv('/content/train.csv', sep='\n')
DATA_TEST = pd.read_csv('/content/test.csv', sep='\n')
DATA_VAL = pd.read_csv('/content/val.csv', sep='\n')

TOKEN_LIST = []
NER_TAG_LIST = []

tmp1_list = []
tmp2_list = []
list_with_seq = []
list_with_ner = []
count_place = []
count = 0

train_df, test_df, val_df  = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

In [66]:
def element_distributor(element):
    element_after_distributor = element.split(' ')
    TOKEN_LIST.append(element_after_distributor[0])
    NER_TAG_LIST.append(element_after_distributor[1])

In [67]:
def function(DATA, df):
  DATA[DATA.columns[0]].apply(element_distributor)
  for token,tag in zip(TOKEN_LIST, NER_TAG_LIST):
      tmp1_list=[]
      tmp2_list=[]
      tmp1_list.append(token)
      tmp2_list.append(tag)
      if token == '.':
          list_with_seq.append(tmp1_list)
          list_with_ner.append(tmp2_list)
  df['ner_tags'] = list_with_ner
  df['tokens'] = list_with_seq
  return df 

In [68]:
DATA_TRAIN['title,intro'].apply(element_distributor)
print('ready')

ready


In [69]:
for token,tag in zip(TOKEN_LIST, NER_TAG_LIST):
  tmp1_list.append(token)
  tmp2_list.append(tag)
  if token == '.':
    list_with_seq.append(tmp1_list)
    list_with_ner.append(tmp2_list)
    tmp1_list=[]
    tmp2_list=[]
    

In [70]:
train_df['ner_tags'] = list_with_ner
train_df['tokens'] = list_with_seq

In [71]:
TOKEN_LIST = []
NER_TAG_LIST = []

tmp1_list = []
tmp2_list = []
list_with_seq = []
list_with_ner = []

In [72]:
DATA_TEST['title,intro'].apply(element_distributor)
print('ready')

ready


In [73]:
for token,tag in zip(TOKEN_LIST, NER_TAG_LIST):
  tmp1_list.append(token)
  tmp2_list.append(tag)
  if token == '.':
    list_with_seq.append(tmp1_list)
    list_with_ner.append(tmp2_list)
    tmp1_list=[]
    tmp2_list=[]

In [74]:
test_df['ner_tags'] = list_with_ner
test_df['tokens'] = list_with_seq

In [75]:
TOKEN_LIST = []
NER_TAG_LIST = []

tmp1_list = []
tmp2_list = []
list_with_seq = []
list_with_ner = []

In [76]:
DATA_VAL['title,intro'].apply(element_distributor)
print('ready')

ready


In [77]:
for token,tag in zip(TOKEN_LIST, NER_TAG_LIST):
  tmp1_list.append(token)
  tmp2_list.append(tag)
  if token == '.':
    list_with_seq.append(tmp1_list)
    list_with_ner.append(tmp2_list)
    tmp1_list=[]
    tmp2_list=[]

In [78]:
val_df['ner_tags'] = list_with_ner
val_df['tokens'] = list_with_seq

# Datasets

In [79]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

In [80]:
from datasets.features import ClassLabel

features = train_dataset.features
column_names = train_dataset.column_names
text_column_name = "tokens"
label_column_name = "ner_tags"

def get_label_list(labels):
    unique_labels = set()
    for label in labels:
        unique_labels = unique_labels | set(label)
    label_list = list(unique_labels)
    label_list.sort()
    return label_list

if isinstance(features[label_column_name].feature, ClassLabel):
    label_list = features[label_column_name].feature.names
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
else:
    label_list = get_label_list(train_dataset[label_column_name])
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
num_labels = len(label_list)

In [81]:
label2id

{'B-LOC': 0,
 'B-ORG': 1,
 'B-PER': 2,
 'I-LOC': 3,
 'I-ORG': 4,
 'I-PER': 5,
 'O': 6}

In [82]:
def conver_label_to_id(text_labels):
  id_list = []
  for label in text_labels:
    for k,v in label2id.items():
      if  k == label:
        id_list.append(v)
  text_labels = id_list
  return  text_labels

In [83]:
train_df['ner_tags'] = train_df['ner_tags'].apply(conver_label_to_id)

In [84]:
test_df['ner_tags'] = test_df['ner_tags'].apply(conver_label_to_id)

In [85]:
val_df['ner_tags'] = val_df['ner_tags'].apply(conver_label_to_id)

In [86]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

In [87]:
val_dataset['ner_tags'][0]

[6, 6, 6, 1, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 0, 6]

In [88]:
model_checkpoint = "bert-base-cased"

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large-finetuned-conll03-english")

#model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-large-finetuned-conll03-english")

In [ ]:
inputs = tokenizer(train_dataset["tokens"][0],is_split_into_words=True)
inputs.tokens()

In [91]:
word_ids = inputs.word_ids()

In [92]:
labels = train_dataset["ner_tags"][0]

In [93]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for i in word_ids:
        if i == None:
          new_labels.append(-100)
        else:
          new_labels.append(labels[i])

    return new_labels

In [94]:
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[6, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 4, 6, 6, 6, 6, 6, 6, 6, 6, 2, 6]
[-100, 6, 6, 2, 2, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 2, 6, 6, -100]


In [95]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [96]:
tokenized_train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)

  0%|          | 0/8 [00:00<?, ?ba/s]

In [97]:
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [98]:
tokenized_val_dataset = val_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [99]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [100]:
batch = data_collator([tokenized_train_dataset[i] for i in range(2)])
batch["labels"]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    6,    6,    2,    2,    2,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    1,    1,    4,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    2,    2,    6,    6, -100],
        [-100,    6,    6,    1,    1,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    2,    2,    2,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    0,    0,    0,    0,    0,    6,    6,    6, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [101]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [102]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [103]:
import evaluate

metric = evaluate.load("seqeval")

In [104]:
ner_feature = train_dataset.features["ner_tags"]
ner_feature

Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)

In [105]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [106]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-LOC",
    "1": "B-ORG",
    "2": "B-PER",
    "3": "I-LOC",
    "4": "I-ORG",
    "5": "I-PER",
    "6": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 0,
    "B-ORG": 1,
    "B-PER": 2,
    "I-LOC": 3,
    "I-ORG": 4,
    "I-PER": 5,
    "O": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "trans

In [107]:
model.config.num_labels

7

In [108]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [109]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [113]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [123]:
!pip install -q keras

In [125]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

RuntimeError: ignored

In [122]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

RuntimeError: ignored